# Data preparation and infrastructure exposure to flooding

This notebook forms the basis of "Hands-On 5" in the CCG course.

1. Extract infrastructure data from OpenStreetMap
2. Extract flood hazard data from Aqueduct
3. Intersect floods with roads to calculate exposure
4. Open QGIS to look at the data

## Activity 1: Extract infrastructure data

### Step 1) On your desktop, create a folder called `ghana_tutorial`

### Step 2) Create a variable to store the folder location

In the cell below, type in the path to your desktop, by changing NAME to match your username as shown on your computer

In [2]:
# edit this if using a Mac (otherwise delete)
data_folder = "/Users/NAME/Desktop/ghana_tutorial"

# edit this if using Windows (otherwise delete)
data_folder = "C:\\Users\\NAME\\Desktop\\ghana_tutorial"

# delete this line
data_folder = "../data"

### Step 3) Load Python libraries

In [3]:
# The os and subprocess modules are built into Python
# see https://docs.python.org/3/library/os.html
import os 
# see https://docs.python.org/3/library/subprocess.html
import subprocess

# Pandas and GeoPandas are libraries for working with datasets
# see https://geopandas.org/
import geopandas as gpd
gpd._compat.USE_PYGEOS = False
# see https://pandas.pydata.org/
import pandas as pd 
# snkit helps generate connected networks from lines and nodes
# see https://snkit.readthedocs.io/
import snkit
import snkit.network

# PyPROJ is a library for working with geographic projections 
# see https://pyproj4.github.io/
from pyproj import Geod

### Step 4) and 5) Download and save data

Download the `ghana-latest-free.shp.zip` dataset from http://download.geofabrik.de/africa/ghana.html, extract the zip folder and save the extracted folder within your new folder `ghana_tutorial`

### Step 6) Load the road dataset you've just downloaded

In [4]:
roads = gpd.read_file(
    os.path.join(data_folder, 'ghana-latest-free.shp', 'gis_osm_roads_free_1.shp'))

### Step 7) To take a look at the data and the attribute table fill in and run the next two cells

In [18]:
roads

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,4790591,5121,unclassified,None,None,F,0,0,F,F,"LINESTRING (-0.17184 5.60847, -0.17182 5.60849..."
1,4790592,5122,residential,Nortei Ababio Road,None,B,0,0,F,F,"LINESTRING (-0.18282 5.61197, -0.18336 5.61198..."
2,4790594,5115,tertiary,None,None,F,0,0,F,F,"LINESTRING (-0.17544 5.60550, -0.17418 5.60555..."
3,4790596,5121,unclassified,None,None,F,0,0,F,F,"LINESTRING (-0.17207 5.60853, -0.17207 5.60844..."
4,4790597,5122,residential,Volta Road,None,B,0,0,F,F,"LINESTRING (-0.18282 5.61197, -0.18280 5.61262..."
...,...,...,...,...,...,...,...,...,...,...,...
295578,921056069,5154,path,None,None,B,0,0,F,F,"LINESTRING (-2.49301 9.10215, -2.49288 9.10219..."
295579,921056070,5154,path,None,None,B,0,0,F,F,"LINESTRING (-2.49266 9.10205, -2.49261 9.10199..."
295580,921056133,5154,path,None,None,B,0,0,F,F,"LINESTRING (-2.48456 9.09706, -2.48471 9.09713..."
295581,921056135,5154,path,None,None,B,0,0,F,F,"LINESTRING (-2.48496 9.09760, -2.48487 9.09761..."


In [19]:
roads.fclass.unique()

array(['unclassified', 'residential', 'tertiary', 'tertiary_link',
       'secondary', 'primary', 'service', 'trunk', 'primary_link',
       'secondary_link', 'trunk_link', 'footway', 'path', 'track',
       'motorway', 'motorway_link', 'track_grade3', 'track_grade4',
       'steps', 'pedestrian', 'bridleway', 'unknown', 'cycleway',
       'track_grade2', 'track_grade5', 'track_grade1', 'living_street'],
      dtype=object)

### Step 8) Next we want to make a couple of changes to the data

Filter out minor and residential roads, tracks and paths.

In [5]:
# Keep only the specified columns
roads = roads[['osm_id', 'fclass', 'name', 'geometry']]
# Keep only the roads whose "fclass" is in the list
roads = roads[
    roads.fclass.isin([
        'motorway',
        'motorway_link',
        'trunk',
        'trunk_link',
        'primary', 
        'primary_link',
        'secondary', 
        'secondary_link', 
        'tertiary',
        'tertiary_link'
    ])
]
# Rename some columns
roads = roads.rename(
    columns={
        'fclass': 'road_type',
    })

Create topological network information - this adds information that will let us find routes over the road network.
- add nodes at the start and end of each road segment
- split roads at junctions, so each segment goes from junction to junction
- add ids to each node and edge, and add `from_id` and `to_id` to each edge

In [6]:
road_network = snkit.Network(edges=roads)

In [7]:
with_endpoints = snkit.network.add_endpoints(road_network)
split_edges = snkit.network.split_edges_at_nodes(with_endpoints)
with_ids = snkit.network.add_ids(split_edges, id_col='id', edge_prefix='roade', node_prefix='roadn')
connected = snkit.network.add_topology(with_ids)
roads = connected.edges
road_nodes = connected.nodes

Calculate the length of each road segment in meters

In [8]:
geod = Geod(ellps='WGS84')
roads['length_m'] = roads.geometry.apply(geod.geometry_length)

In [9]:
roads.tail()

,osm_id,road_type,name,geometry,id,from_id,to_id,length_m
14579,909292442,trunk,Kumasi - Techiman,"LINESTRING (-1.64543 6.75849, -1.64545 6.75834)",roade_14579,roadn_3093,roadn_11198,16.851744
14580,911591641,tertiary,None,"LINESTRING (-1.57623 6.70866, -1.57637 6.70861)",roade_14580,roadn_11199,roadn_4529,16.645715
14581,911591642,tertiary,None,"LINESTRING (-1.57122 6.71057, -1.57141 6.71047...",roade_14581,roadn_11200,roadn_11199,593.614781
14582,914013514,tertiary,None,"LINESTRING (-1.93890 6.31313, -1.93897 6.31309...",roade_14582,roadn_11201,roadn_11202,2729.980925
14583,914269553,tertiary,None,"LINESTRING (-2.02465 7.23432, -2.02483 7.23514...",roade_14583,roadn_11203,roadn_11204,1256.109858


In [15]:
roads.crs = {'init': 'epsg:4326'}
road_nodes.crs = {'init': 'epsg:4326'}

### Step 9) Save the pre-processed dataset

In [16]:
roads.to_file(
    os.path.join(data_folder, 'GHA_OSM_roads.gpkg'),
    layer='edges', 
    driver="GPKG")
road_nodes.to_file(
    os.path.join(data_folder, 'GHA_OSM_roads.gpkg'),
    layer='nodes', 
    driver="GPKG")

## Activity 2: Extract and polygonise hazard data

### Step 1) Download flood hazard data from Aqueduct

The full [Aqueduct dataset](https://www.wri.org/resources/data-sets/aqueduct-floods-hazard-maps) is available to download openly. There are some scripts and summary of the data you may find useful at [nismod/aqueduct](https://github.com/nismod/aqueduct).

There are almost 700 files in the full Aqueduct dataset, of up to around 100MB each, so we don't recommend downloading all of them unless you intend to do further analysis.

For later tutorials, we provide a preprocessed set of hazard polygons for the Ghana example. 

The next steps show how to clip a region out of the global dataset and polygonise it, in case you wish to reproduce this analysis in another part of the world.

For now, we suggest downloading [inunriver_historical_000000000WATCH_1980_rp00100.tif](http://wri-projects.s3.amazonaws.com/AqueductFloodTool/download/v2/inunriver_historical_000000000WATCH_1980_rp00100.tif) to work through the next steps. Save the downloaded file in a new folder titled `flood_layer` under your data_folder.

### Step 2) Run the code below to polygonise the tif files

This converts the flood maps from *tiff files (raster data)* into *shape files (vector data)*. It will take a little time to run.

In [14]:
xmin = "-3.262509"
ymin = "4.737128"
xmax = "1.187968"
ymax = "11.162937"

for root, dirs, files in os.walk(data_folder, 'flood_layer'): 
    print("Looking in", root) 
    for file in sorted(files): 
        if file.endswith(".tif") and not file.endswith("p.tif"): 
            print("Found tif file", file)
            stem = file[:-4]
            input_file = os.path.join(root, file) 
            
            # Clip file to bounds
            clip_file = os.path.join(root, f"{stem}_clip.tif")
            try:
                os.remove(clip_file)
            except FileNotFoundError:
                pass
            p = subprocess.run([
                "gdalwarp", "-te", xmin, ymin, xmax, ymax, input_file, clip_file],
                capture_output=True)
            print(p.stdout.decode('utf8'))
            print(p.stderr.decode('utf8'))
            print(clip_file)

            # Create vector outline of raster areas
            # note that this rounds the floating-point values of flood depth from 
            # the raster to the nearest integer in the vector outlines
            polygons_file = os.path.join(root, f"{stem}.gpkg") 
            try:
                os.remove(polygons_file)
            except FileNotFoundError:
                pass
            p = subprocess.run([
                "gdal_polygonize.py", clip_file,'-q','-f', 'GPKG', polygons_file],
                capture_output=True)
            print(p.stdout.decode('utf8'))
            print(p.stderr.decode('utf8'))  
            print(polygons_file)

Looking in ../data
Looking in ../data/results
Looking in ../data/ghana-latest-free.shp
Looking in ../data/ne_10m_admin_0_countries
Looking in ../data/flood_layer
Found tif file inunriver_historical_000000000WATCH_1980_rp00010.tif
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/inunriver_historical_000000000WATCH_1980_rp00010.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image ../data/flood_layer/inunriver_historical_000000000WATCH_1980_rp00010.tif.
Copying nodata values from source ../data/flood_layer/inunriver_historical_000000000WATCH_1980_rp00010.tif to destination ../data/flood_layer/inunriver_historical_000000000WATCH_1980_rp00010_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/inunriver_historical_000000000WATCH_1980_rp00010_clip.tif


../data/flood_layer/inunriver_historical_000000000WATCH_1980_rp00010.gpkg
Found tif file inunriver_historical_000000000WATCH_1980_rp00100.tif
Creating output file tha



../data/flood_layer/inunriver_rcp4p5_0000HadGEM2-ES_2080_rp01000.gpkg
Found tif file inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00010.tif
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00010.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image ../data/flood_layer/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00010.tif.
Copying nodata values from source ../data/flood_layer/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00010.tif to destination ../data/flood_layer/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00010_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00010_clip.tif


../data/flood_layer/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00010.gpkg
Found tif file inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00100.tif
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp00100.tif [1/1] : 0Using internal 



../data/flood_layer/inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp01000.gpkg
Found tif file inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00010.tif
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00010.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image ../data/flood_layer/inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00010.tif.
Copying nodata values from source ../data/flood_layer/inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00010.tif to destination ../data/flood_layer/inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00010_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00010_clip.tif


../data/flood_layer/inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00010.gpkg
Found tif file inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00100.tif
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/inunriver_rcp8p5_0000HadGEM2-ES_2080_rp00100.tif [1/1] : 0Using internal 



../data/flood_layer/threshold/inunriver_historical_000000000WATCH_1980_rp00010_2.0m999.0m.gpkg
Found tif file inunriver_historical_000000000WATCH_1980_rp00100_1.0m999.0m.tif
Copying color table from ../data/flood_layer/threshold/inunriver_historical_000000000WATCH_1980_rp00100_1.0m999.0m.tif to new file.
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/threshold/inunriver_historical_000000000WATCH_1980_rp00100_1.0m999.0m.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ../data/flood_layer/threshold/inunriver_historical_000000000WATCH_1980_rp00100_1.0m999.0m.tif.
Copying nodata values from source ../data/flood_layer/threshold/inunriver_historical_000000000WATCH_1980_rp00100_1.0m999.0m.tif to destination ../data/flood_layer/threshold/inunriver_historical_000000000WATCH_1980_rp00100_1.0m999.0m_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/threshold/inunriver_historical_000000000WATCH_1980_rp00100_1.0m999.0



../data/flood_layer/threshold/inunriver_rcp4p5_00000NorESM1-M_2080_rp01000_1.0m999.0m.gpkg
Found tif file inunriver_rcp4p5_00000NorESM1-M_2080_rp01000_2.0m999.0m.tif
Copying color table from ../data/flood_layer/threshold/inunriver_rcp4p5_00000NorESM1-M_2080_rp01000_2.0m999.0m.tif to new file.
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/threshold/inunriver_rcp4p5_00000NorESM1-M_2080_rp01000_2.0m999.0m.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ../data/flood_layer/threshold/inunriver_rcp4p5_00000NorESM1-M_2080_rp01000_2.0m999.0m.tif.
Copying nodata values from source ../data/flood_layer/threshold/inunriver_rcp4p5_00000NorESM1-M_2080_rp01000_2.0m999.0m.tif to destination ../data/flood_layer/threshold/inunriver_rcp4p5_00000NorESM1-M_2080_rp01000_2.0m999.0m_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/threshold/inunriver_rcp4p5_00000NorESM1-M_2080_rp01000_2.0m999.0m_clip.tif


../data/flood_layer

...



../data/flood_layer/threshold/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp01000_1.0m999.0m.gpkg
Found tif file inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp01000_2.0m999.0m.tif
Copying color table from ../data/flood_layer/threshold/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp01000_2.0m999.0m.tif to new file.
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/threshold/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp01000_2.0m999.0m.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ../data/flood_layer/threshold/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp01000_2.0m999.0m.tif.
Copying nodata values from source ../data/flood_layer/threshold/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp01000_2.0m999.0m.tif to destination ../data/flood_layer/threshold/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp01000_2.0m999.0m_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/threshold/inunriver_rcp4p5_00IPSL-CM5A-LR_2080_rp01000_2.0m999.0m_clip.tif


../data/flood_layer



../data/flood_layer/threshold/inunriver_rcp8p5_00000NorESM1-M_2080_rp00010_2.0m999.0m.gpkg
Found tif file inunriver_rcp8p5_00000NorESM1-M_2080_rp00100_1.0m999.0m.tif
Copying color table from ../data/flood_layer/threshold/inunriver_rcp8p5_00000NorESM1-M_2080_rp00100_1.0m999.0m.tif to new file.
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/threshold/inunriver_rcp8p5_00000NorESM1-M_2080_rp00100_1.0m999.0m.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ../data/flood_layer/threshold/inunriver_rcp8p5_00000NorESM1-M_2080_rp00100_1.0m999.0m.tif.
Copying nodata values from source ../data/flood_layer/threshold/inunriver_rcp8p5_00000NorESM1-M_2080_rp00100_1.0m999.0m.tif to destination ../data/flood_layer/threshold/inunriver_rcp8p5_00000NorESM1-M_2080_rp00100_1.0m999.0m_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/threshold/inunriver_rcp8p5_00000NorESM1-M_2080_rp00100_1.0m999.0m_clip.tif


../data/flood_layer



../data/flood_layer/threshold/inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp01000_1.0m999.0m.gpkg
Found tif file inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp01000_2.0m999.0m.tif
Copying color table from ../data/flood_layer/threshold/inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp01000_2.0m999.0m.tif to new file.
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/threshold/inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp01000_2.0m999.0m.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ../data/flood_layer/threshold/inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp01000_2.0m999.0m.tif.
Copying nodata values from source ../data/flood_layer/threshold/inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp01000_2.0m999.0m.tif to destination ../data/flood_layer/threshold/inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp01000_2.0m999.0m_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/threshold/inunriver_rcp8p5_0000GFDL-ESM2M_2080_rp01000_2.0m999.0m_clip.tif


../data/flood_layer



../data/flood_layer/threshold/inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00010_2.0m999.0m.gpkg
Found tif file inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00100_1.0m999.0m.tif
Copying color table from ../data/flood_layer/threshold/inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00100_1.0m999.0m.tif to new file.
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/threshold/inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00100_1.0m999.0m.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ../data/flood_layer/threshold/inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00100_1.0m999.0m.tif.
Copying nodata values from source ../data/flood_layer/threshold/inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00100_1.0m999.0m.tif to destination ../data/flood_layer/threshold/inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00100_1.0m999.0m_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/threshold/inunriver_rcp8p5_00IPSL-CM5A-LR_2080_rp00100_1.0m999.0m_clip.tif


../data/flood_layer



../data/flood_layer/threshold/inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000_1.0m999.0m.gpkg
Found tif file inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000_2.0m999.0m.tif
Copying color table from ../data/flood_layer/threshold/inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000_2.0m999.0m.tif to new file.
Creating output file that is 534P x 771L.
Processing ../data/flood_layer/threshold/inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000_2.0m999.0m.tif [1/1] : 0Using internal nodata values (e.g. 0) for image ../data/flood_layer/threshold/inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000_2.0m999.0m.tif.
Copying nodata values from source ../data/flood_layer/threshold/inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000_2.0m999.0m.tif to destination ../data/flood_layer/threshold/inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000_2.0m999.0m_clip.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


../data/flood_layer/threshold/inunriver_rcp8p5_MIROC-ESM-CHEM_2080_rp01000_2.0m999.0m_clip.tif


../data/flood_layer

## Activity 3: Intersect hazard 

Let us now intersect the hazard and the roads, starting with one hazard initially so we save time.

### Step 1) Specify your input and output path as well as the name of the intersection

In [43]:
flood_path = os.path.join(
    data_folder, 
    'flood_layer', 
    'inunriver_historical_000000000WATCH_1980_rp00100.gpkg')

output_path = os.path.join(
    data_folder, 
    'results', 
    'inunriver_historical_000000000WATCH_1980_rp00100_exposure.gpkg')

flood = gpd.read_file(flood_path) \
    .rename(columns={'DN': 'depth_m'})
flood = flood[flood.depth_m > 0]

/home/tom/miniconda3/lib/python3.7/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


### Step 2) Run the intersection

In [44]:
flood_intersections = gpd.overlay(GHA_OSM_roads, flood, how='intersection')
flood_intersections

,id,osm_id,road_type,name,length_m,depth_m,geometry
0,30,4845650,trunk,None,256.660267,9,"LINESTRING (-1.16109 9.14004, -1.15927 9.14149)"
1,322,29983106,trunk,Tamale - Techiman Road,43306.482460,9,"LINESTRING (-1.16109 9.14004, -1.16150 9.13968..."
2,8331,627211273,trunk,Tamale - Techiman Road,32890.731018,9,"LINESTRING (-1.15927 9.14149, -1.15867 9.14195..."
3,73,11180537,trunk,Winneba Road,3134.738925,1,"LINESTRING (-0.31338 5.55362, -0.31494 5.55356..."
4,156,12696453,trunk_link,George Walker Bush Highway,426.151625,1,"LINESTRING (-0.28295 5.57111, -0.28287 5.57112..."
...,...,...,...,...,...,...,...
1945,10545,863568484,tertiary,None,9837.877138,8,"LINESTRING (-0.20284 7.70417, -0.20320 7.70362..."
1946,10584,891519151,trunk,None,4813.738570,23,"LINESTRING (0.06023 6.15397, 0.06017 6.15387, ..."
1947,10584,891519151,trunk,None,4813.738570,18,"LINESTRING (0.05451 6.14302, 0.05441 6.14278, ..."
1948,10584,891519151,trunk,None,4813.738570,22,"LINESTRING (0.05562 6.14564, 0.05528 6.14480, ..."


Calculate the exposed length

In [45]:
geod = Geod(ellps='WGS84')
flood_intersections['flood_length_m'] = flood_intersections.geometry.apply(geod.geometry_length)

In [46]:
flood_intersections.tail()

,id,osm_id,road_type,name,length_m,depth_m,geometry,flood_length_m
1945,10545,863568484,tertiary,None,9837.877138,8,"LINESTRING (-0.20284 7.70417, -0.20320 7.70362...",215.740629
1946,10584,891519151,trunk,None,4813.738570,23,"LINESTRING (0.06023 6.15397, 0.06017 6.15387, ...",1072.970524
1947,10584,891519151,trunk,None,4813.738570,18,"LINESTRING (0.05451 6.14302, 0.05441 6.14278, ...",1685.781958
1948,10584,891519151,trunk,None,4813.738570,22,"LINESTRING (0.05562 6.14564, 0.05528 6.14480, ...",314.923301
1949,10584,891519151,trunk,None,4813.738570,16,"LINESTRING (0.04861 6.12897, 0.04854 6.12877, ...",933.468177


Calculate the proportion of roads in our dataset which are exposed to >=1m flood depths in this scenario

In [47]:
exposed_length = flood_intersections.flood_length_m.sum()
exposed_length

1635374.9644959064

In [48]:
all_roads_in_dataset_length = GHA_OSM_roads.length_m.sum()
all_roads_in_dataset_length

28720890.745240472

In [49]:
proportion = exposed_length / all_roads_in_dataset_length
proportion

0.056940259235062726

In [50]:
f"{proportion:.0%} of roads in this dataset are exposed to flood depths of >= 1m in a historical 1-in-100 year flood"

'6% of roads in this dataset are exposed to flood depths of >= 1m in a historical 1-in-100 year flood'

Save to file (with spatial data)

In [40]:
flood_intersections.to_file(output_path, driver="GPKG")

In [33]:
flood_intersections

,id,osm_id,road_type,name,length_m,depth_m,geometry,flood_length_m
0,30,4845650,trunk,None,256.660267,2,"LINESTRING (-1.16109 9.14004, -1.15927 9.14149)",256.660267
1,322,29983106,trunk,Tamale - Techiman Road,43306.482460,2,"LINESTRING (-1.16109 9.14004, -1.16150 9.13968...",158.781284
2,8331,627211273,trunk,Tamale - Techiman Road,32890.731018,2,"LINESTRING (-1.15927 9.14149, -1.15867 9.14195...",705.764920
3,322,29983106,trunk,Tamale - Techiman Road,43306.482460,5,"LINESTRING (-1.16228 9.13924, -1.16237 9.13920...",2631.818501
4,322,29983106,trunk,Tamale - Techiman Road,43306.482460,2,"LINESTRING (-1.18729 9.13615, -1.18871 9.13516...",1169.962200
...,...,...,...,...,...,...,...,...
817,10545,863568484,tertiary,None,9837.877138,3,"LINESTRING (-0.20284 7.70417, -0.20320 7.70362...",215.740629
818,10584,891519151,trunk,None,4813.738570,23,"LINESTRING (0.06023 6.15397, 0.06017 6.15387, ...",1072.970524
819,10584,891519151,trunk,None,4813.738570,18,"LINESTRING (0.05451 6.14302, 0.05441 6.14278, ...",1685.781958
820,10584,891519151,trunk,None,4813.738570,22,"LINESTRING (0.05562 6.14564, 0.05528 6.14480, ...",314.923301


Save to CSV (without spatial data)

In [39]:
flood_intersections.drop(columns='geometry').to_csv(output_path.replace(".gpkg", ".csv"))